In [86]:
import numpy as np
import math
from sklearn import model_selection

In [87]:
dataset = np.loadtxt("cleaned_titanic.csv")
x = dataset[:,0:-1]
y = dataset[:,-1]
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y)
data = np.append(x_train, y_train.reshape(-1,1), axis=1)

In [88]:
# Function to calculate gini_index
def gini_index(y):
    p2 = 0
    for i in set(y):
        pi = y.tolist().count(i)/len(y)
        p2 += pi**2
    return 1 - p2

In [89]:
# For Creating the Decision Tree
def decTreeCreate(data, fs, ofs, tree, indx):
    x = data[:,0:-1]
    y = data[:,-1]
    if len(fs) == 0:
        return
    
    if len(set(y)) == 1:
        return
    
    # Current gini index
    cur_gini = gini_index(y)
    
    l = np.zeros(len(fs)) # List for storing delta gini index
    for i in range(len(fs)):
        for j in set(x[:,i]):
            y_split = data[np.where(data[:,i] == j)][:,-1]
            split_gini = gini_index(y_split)
            l[i] += len(y_split)/len(y) * split_gini
        l[i] = cur_gini - l[i]
    b = l.tolist().index(max(l))
    o_b = ofs.index(fs[b])
    # Remove the feature on which split was done
    fs1 = fs.copy()
    fs1.remove(fs1[b])
    
    # All classes of values in x for selected feature
    f_distinct_values = list(set(x[:,b]))
    n = len(f_distinct_values)
    
    # Split number 1
    # Contains half the classes of values in x for feature selected above 
    data_split1 = data[np.where(data[:,b] == f_distinct_values[0])]
    for j in range(1, n//2 + 1):
        split = data[np.where(data[:,b] == f_distinct_values[j])]
        np.append(data_split1, split)
    
    # Remove the selected feature form x 
    data_split1 = np.delete(data_split1, b, axis=1)
    y_split1 = data_split1[:,-1]
    
    # First child node for current node
    """ Node contains 
        1.feature: index of feature on which split was done
        2.feature_values: classes of values in x for 'feature'
        3.pred: Value that could be predicted for current node
        4.final: 1 if leaf node , 0 otherwise
    """
    node1 = {
        'feature': o_b,
        'feature_values': f_distinct_values[0:n//2+1],
    }
    z = 0
    p_i = y_split1[0]
    for i in set(y_split1):
        if y_split1.tolist().count(i) > z:
            z = y_split1.tolist().count(i)
            p_i = i
    node1['pred'] = p_i
    node1['final'] = 0
    if len(set(y_split1)) == 1:
        node1['pred'] = y_split1[0]
        node1['final'] = 1
    elif len(fs) == 0:
        node1['final'] = 1

    # Split number 2
    data_split2 = data[np.where(data[:,b] == f_distinct_values[n//2])]
    for j in range(n//2+1, n):
        split = data[np.where(data[:,b] == f_distinct_values[j])]
        np.append(data_split2, split)
        
    data_split2 = np.delete(data_split2, b, axis=1)
    y_split2 = data_split2[:,-1]
    node2 = {
        'feature': o_b,
        'feature_values': f_distinct_values[n//2:n]
    }
    z = 0
    p_i = y_split2[0]
    for i in set(y_split2):
        if y_split2.tolist().count(i) > z:
            x = y_split2.tolist().count(i)
            p_i = i
    node2['pred'] = p_i
    node2['final'] = 0
    if len(set(y_split2)) == 1:
        node2['pred'] = y_split2[0]
        node2['final'] = 1
    elif len(fs) == 0:
        node2['final'] = 1

    # Add the nodes in tree
    tree[indx] = node1
    tree[indx+1] = node2
    
    # Make recursive calls for both nodes
    decTreeCreate(data_split1, fs1, ofs, tree, 2*indx)
    decTreeCreate(data_split2, fs1, ofs, tree, 2*(indx + 1))

In [90]:
# List of features
fs = ['Pclass','Sex','Age','SibSp','a','b']
fs

['Pclass', 'Sex', 'Age', 'SibSp', 'a', 'b']

In [91]:
# Implementing tree using arrays
# children for a node at position i will be at 2*i and 2*i + 1
n = 2**(len(fs)+1)
tree = np.zeros(n).tolist() # Complete binary tree
decTreeCreate(data, fs, fs, tree, 2)
tree

[0.0,
 0.0,
 {'feature': 1, 'feature_values': [1.0, 2.0], 'final': 0, 'pred': 0.0},
 {'feature': 1, 'feature_values': [2.0], 'final': 0, 'pred': 1.0},
 {'feature': 2, 'feature_values': [1.0, 2.0, 3.0], 'final': 0, 'pred': 1.0},
 {'feature': 2, 'feature_values': [3.0, 4.0], 'final': 0, 'pred': 1.0},
 {'feature': 0, 'feature_values': [1.0, 2.0], 'final': 1, 'pred': 1.0},
 {'feature': 0, 'feature_values': [2.0, 3.0], 'final': 0, 'pred': 1.0},
 {'feature': 3,
  'feature_values': [0.0, 1.0, 2.0, 3.0],
  'final': 1,
  'pred': 1.0},
 {'feature': 3, 'feature_values': [3.0, 4.0, 5.0], 'final': 1, 'pred': 0.0},
 {'feature': 0, 'feature_values': [1.0, 2.0], 'final': 0, 'pred': 0.0},
 {'feature': 0, 'feature_values': [2.0, 3.0], 'final': 0, 'pred': 1.0},
 0.0,
 0.0,
 {'feature': 2, 'feature_values': [1.0, 2.0, 3.0], 'final': 1, 'pred': 1.0},
 {'feature': 2, 'feature_values': [3.0, 4.0], 'final': 1, 'pred': 1.0},
 0.0,
 0.0,
 0.0,
 0.0,
 {'feature': 3, 'feature_values': [0.0, 1.0], 'final': 0, 'pre

In [92]:
# Function to predict for each row of x_test
def predict(x):
    i = 2
    while i < len(tree):
        node = tree[i]
        if x[node['feature']] in node['feature_values']:
            if node['final'] == 1:
                return node['pred']
            elif len(tree)>2*i and type(tree[2*i]) == type(dict()):
                i*=2
            else:
                return node['pred']
        elif type(tree[i+1]) == type(dict()):
            i += 1
        else:
            return node['pred']

In [93]:
y_pred = []
for i in range(0,len(x_test)):
    y_pred.append(predict(x_test[i]))

In [94]:
print(y_pred)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]


In [95]:
#Score
a = ((y_pred-y_test)**2).sum()
print((len(y_test) - a)/len(y_test))

0.455089820359
